In [1]:
%matplotlib qt
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import seaborn as sns
import mne
from nilearn.plotting import plot_connectome
from scipy.spatial.distance import pdist
from scipy.stats import ttest_ind
from sklearn.metrics.pairwise import cosine_similarity
import os
import learn_graph
from tqdm.notebook import tqdm
from pingouin import intraclass_corr

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import MDS
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
import statsmodels.api as sm

/Users/payamsadeghishabestari/virtual_envs/.venv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
###### fig 1_A ######
fname = Path.cwd().parent / "data" / "graph_comparison_2.csv"
df = pd.read_csv(fname)
vals = [0.25, 0.5, 1, 1.5]
df = df.query("alpha in @vals and beta in @vals")
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
g = sns.FacetGrid(df, col="frequency_range", row="alpha", hue="beta", height=2.5, aspect=1.3, palette="YlGnBu_d", legend_out=True)
kwargs = {"markersize": 2.5, "lw": 2}
g.map_dataframe(sns.pointplot, x="times_used", y="PDiv", **kwargs)
g.set(xticklabels=[], title="", ylabel="", xlabel="")
legend_kwargs = {"bbox_to_anchor": (0.5, 0.7, 0.5, 0.5)}
g.add_legend(**legend_kwargs)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "PDiv.pdf")

###### fig 1_B ######
fname = Path.cwd().parent / "data" / "graph_comparison_2.csv"
df = pd.read_csv(fname)
vals = [0.25, 0.5, 1, 1.5]
df = df.query("alpha in @vals and beta in @vals")
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
g = sns.FacetGrid(df, col="frequency_range", row="alpha", hue="beta", height=2.5, aspect=1.3, palette="YlGnBu_d", legend_out=True)
kwargs = {"markersize": 2.5, "lw": 2}
g.map_dataframe(sns.pointplot, x="times_used", y="euc_dist", **kwargs)
g.set(xticklabels=[], title="", ylabel="", xlabel="")
legend_kwargs = {"bbox_to_anchor": (0.5, 0.7, 0.5, 0.5)}
g.add_legend(**legend_kwargs)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "euc_dist.pdf")

###### fig 1_C ######
fname = Path.cwd().parent / "data" / "df_diff.csv"
df = pd.read_csv(fname)
vals = [1, 1.5]
df = df.query("alpha in @vals and beta in @vals")
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
cl1, cl2 = sns.color_palette("YlGnBu_d")[3], sns.color_palette("YlGnBu_d")[5] 
g = sns.FacetGrid(df, col="frequency_range", row="alpha", hue="beta", height=2.5, aspect=1.7, palette=[cl1, cl2], legend_out=True)
kwargs = {"markersize": 2.5, "lw": 2}
g.map_dataframe(sns.pointplot, x="times_used", y="PDiv", **kwargs)
g.set(xticklabels=[], title="", ylabel="", xlabel="")
legend_kwargs = {"bbox_to_anchor": (0.5, 0.6, 0.5, 0.5)}
g.add_legend(**legend_kwargs)
def add_hline(data, **kwargs):
    ax = plt.gca()
    ax.axhline(0, color='r', linestyle='--', linewidth=2)
g.map_dataframe(add_hline)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "PDiv_dist_hline.pdf")

###### fig 1_D ######
fname = Path.cwd().parent / "data" / "df_diff.csv"
df = pd.read_csv(fname)
vals = [1, 1.5]
df = df.query("alpha in @vals and beta in @vals")
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
cl1, cl2 = sns.color_palette("YlGnBu_d")[3], sns.color_palette("YlGnBu_d")[5] 
g = sns.FacetGrid(df, col="frequency_range", row="alpha", hue="beta", height=2.5, aspect=1.7, palette=[cl1, cl2], legend_out=True)
kwargs = {"markersize": 2.5, "lw": 2}
g.map_dataframe(sns.pointplot, x="times_used", y="euc_dist", **kwargs)
g.set(xticklabels=[], title="", ylabel="", xlabel="")
legend_kwargs = {"bbox_to_anchor": (0.5, 0.6, 0.5, 0.5)}
g.add_legend(**legend_kwargs)
def add_hline(data, **kwargs):
    ax = plt.gca()
    ax.axhline(0, color='r', linestyle='--', linewidth=2)
g.map_dataframe(add_hline)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "euc_dist_hline.pdf")

In [ ]:
###### fig 2_A ######
fname = Path.cwd().parent / "data" / "identifiability_no_subtract.pkl"
with open(fname, 'rb') as file:
    im_dict = pickle.load(file)

cmap = sns.diverging_palette(230, 20, as_cmap=True)
fig, axs = plt.subplots(1, 2, figsize=(11, 5))
for ax, title in zip(axs, ["alpha", "gamma"]):
    im_matrix = im_dict[title]["aparc"][1]
    sns.heatmap(im_matrix, cmap=cmap, vmax=1, center=0.5,
                square=True, linewidths=0.5,
                cbar_kws={"shrink": .5}, ax=ax)
    ax.axvline(x=23, ymin=0, ymax=1, color="k", linewidth=2)
    ax.axhline(y=23, xmin=0, xmax=1, color="k", linewidth=2)

    ax.set(xticks=[], yticks=[], title=title)

n_controls = 23
cl1 = sns.cubehelix_palette(10, rot=2.5, light=.7, reverse=True)[9]
cl2 = sns.cubehelix_palette(10, rot=-2*np.pi/10, light=.7, reverse=True)[1]
fig, axs = plt.subplots(1, 2, figsize=(11, 3))
kwargs = {"linewidth": 2}
for ax, title in zip(axs, ["alpha", "gamma"]):
    im_matrix = im_dict[title]["aparc"][1]
    diagonal_values = np.diag(im_matrix)
    off_diagonal_averages = (np.sum(im_matrix, axis=1) - diagonal_values) / (im_matrix.shape[1] - 1)
    im_diff = diagonal_values - off_diagonal_averages
    datas = [im_diff[:n_controls], im_diff[n_controls:]]
    for data, label, color in zip(datas, ["Controls", "Cases"], [cl1, cl2]):
        sns.kdeplot(data=data, color=color, fill=True, common_norm=True, ax=ax, alpha=0.3, label=label, legend=False, **kwargs)
    ax.legend(frameon=False)
    ax.spines[["right", "left", "top"]].set_visible(False)
    ax.set(xlim=[0.1, 0.8], yticks=[], ylabel="", title=title)

fig, axs = plt.subplots(1, 2, figsize=(11, 3))
kwargs = {"linewidth": 2}
for ax, title in zip(axs, ["alpha", "gamma"]):
    im_matrix = im_dict[title]["aparc"][1]
    diagonal_values = np.diag(im_matrix)
    off_diagonal_averages = (np.sum(im_matrix, axis=1) - diagonal_values) / (im_matrix.shape[1] - 1)
    im_diff = diagonal_values - off_diagonal_averages
    datas = [im_diff[:n_controls], im_diff[n_controls:]]
    stat, p_val = ttest_ind(datas[0], datas[1],
                                    permutations=0, random_state=42)
    print(p_val)
    sns.boxplot(data=datas, dodge=True, orient="h", palette=[cl1, cl2], fill=False, width=0.8, gap=0.01, linewidth=2, ax=ax)
    sns.stripplot(data=datas, dodge=True, orient="h", palette=[cl1, cl2], ax=ax)
    ax.legend(frameon=False)
    ax.spines[["right", "left", "top"]].set_visible(False)
    ax.set(xlim=[0.1, 0.8], yticks=[], ylabel="", title=title)

###### fig 2_B ######
fname = Path.cwd().parent / "data" / "ranks.csv"
df = pd.read_csv(fname)

sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
color_1 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[2]
color_2 = sns.cubehelix_palette(10, rot=8.5, light=.7, reverse=True)[8]
g = sns.FacetGrid(df, col="freq_band", row="group", hue="method", height=2.5, palette=[color_1, color_2], aspect=1.7, legend_out=True)
kwargs = {"element": "step", "discrete": True, "multiple": "dodge", "fill": True, "stat": "percent"}
g.map(sns.histplot, "rank", **kwargs)
legend_kwargs = {"bbox_to_anchor": (0.5, 0.6, 0.5, 0.5)}
g.add_legend(**legend_kwargs)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "ranks.pdf")

###### fig 2_C ######
def plot_gaussian_ellipse(mean, cov, ax, n_std=2.0, **kwargs):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    width, height = 2 * n_std * np.sqrt(vals)
    ellip = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    ax.add_patch(ellip)

for title in ["alpha", "gamma"]:   
    im_matrix = im_dict[title]["aparc"][1]
    s_col = cosine_similarity(im_matrix.T)
    s_row = cosine_similarity(im_matrix)
    sim_matrix = 0.5 * (s_col + s_row)
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=0)
    X_2d = mds.fit_transform(1 - sim_matrix)
    diagonal_values = np.diag(im_matrix)
    off_diagonal_averages = (np.sum(im_matrix, axis=1) - diagonal_values) / (im_matrix.shape[1] - 1)
    im_diff = diagonal_values - off_diagonal_averages
    spectral = SpectralClustering(n_clusters=3, affinity="precomputed", random_state=42)
    clusters = spectral.fit_predict(sim_matrix)

    cl1 = sns.color_palette("blend:#7AB,#EDA", as_cmap=False).as_hex()[3]
    cl2 = sns.color_palette("ch:s=.25,rot=-.25", as_cmap=False).as_hex()[3]
    cl3 = sns.color_palette("dark:#5A9_r", as_cmap=False).as_hex()[3]
    color_mapping = {0: cl1, 1: cl2, 2: cl3}
    palette_color = ['#1f77b4', '#d62728'] 
    colors = ['#1f77b4'] * 23 + ['#d62728'] * 18
    colors_clu = np.array([color_mapping[value] for value in clusters])
    fig, axs = plt.subplots(2, 1, figsize=(3.5, 7), layout="constrained") 
    sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1], hue=colors, size=im_diff, sizes=(20, 400), palette=palette_color, legend=False, ax=axs[0])
    sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1], hue=colors_clu, size=im_diff, sizes=(20, 500), palette=[cl1, cl2, cl3], legend=False, ax=axs[1])

    cl1_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cl1, markersize=10, label='Cluster 1')
    cl2_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cl2, markersize=10, label='Cluster 2')
    cl3_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cl3, markersize=10, label='Cluster 3')
    cl4_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=palette_color[0], markersize=10, label='Controls')
    cl5_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=palette_color[1], markersize=10, label='Cases')
    [ax.spines[["top", "right"]].set_visible(False) for ax in axs]

    for cluster, cl in zip(np.unique(clusters), [cl3, cl2, cl1]):
        cluster_points = X_2d[clusters == cluster]
        mean = np.mean(cluster_points, axis=0)
        cov = np.cov(cluster_points.T)
        plot_gaussian_ellipse(mean, cov, axs[1], n_std=2, alpha=0.12, color=cl)

    fig.savefig(Path.cwd().parent / "figures" / f"clusters_gl_{title}.pdf")

###### fig 2_D ######
fname = Path.cwd().parent / "data" / "identifiability_pli.pkl"
with open(fname, 'rb') as file:
    im_pli_dict = pickle.load(file)

palette_color = ['#1f77b4', '#d62728'] 
colors = ['#1f77b4'] * 23 + ['#d62728'] * 18

fig, axs = plt.subplots(2, 1, figsize=(3.5, 7), layout="constrained") # 
for title, ax in zip(["alpha", "gamma"], axs):

    im_matrix = im_pli_dict[title]
    s_col = cosine_similarity(im_matrix.T)
    s_row = cosine_similarity(im_matrix)
    sim_matrix = 0.5 * (s_col + s_row)
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    X_2d = mds.fit_transform(1 - sim_matrix)
    diagonal_values = np.diag(im_matrix)
    off_diagonal_averages = (np.sum(im_matrix, axis=1) - diagonal_values) / (im_matrix.shape[1] - 1)
    im_diff = diagonal_values - off_diagonal_averages

    sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1], hue=colors, size=im_diff, sizes=(20, 400),
                    palette=palette_color, legend=False, ax=ax)
    ax.set_ylim(-0.9, 0.9)
    ax.spines[["top", "right"]].set_visible(False)

fig.savefig(Path.cwd().parent / "figures" / "clusters_pli.pdf")

In [ ]:
###### figure 3_A ###### (should be updated)
label_names = ["parahippocampal-lh", "superiorfrontal-lh", "superiorfrontal-rh",
                "bankssts-lh", "rostralmiddlefrontal-lh", "frontalpole-lh",
                "lateraloccipital-lh", "parsorbitalis-lh", "frontalpole-rh"] 

brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))
lb_names = [lb.name for lb in brain_labels]
idxs = np.array([lb_names.index(lb) for lb in label_names])
lbs = list(brain_labels[idxs])
cl1 = sns.cubehelix_palette(10, rot=2.5, light=.7, reverse=True)[9]
cl2 = sns.cubehelix_palette(10, rot=-2*np.pi/10, light=.7, reverse=True)[1]
cl3 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[3]
cl4 = list(sns.color_palette("Reds")[-1])
cl5 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[2]
cl6 = sns.cubehelix_palette(10, rot=8.5, light=.7, reverse=True)[8]
cl7 = sns.cubehelix_palette(10, rot=8.5, light=.7, reverse=True)[0]
cl8 = sns.cubehelix_palette(10, rot=1.5, light=.7, reverse=True)[2]
cl9 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[5]
colors = [cl1, cl2, cl3, cl4, cl5, cl6, cl7, cl8, cl9]

brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="lateral", **brain_kwargs)
for lb, lb_name, color in zip(lbs, label_names, colors):
    if lb.hemi == "lh":
        brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

brain_scr_1 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="lateral", **brain_kwargs)
for lb, lb_name, color in zip(lbs, label_names, colors):
    if lb.hemi == "rh":
        brain_scr_2 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)

brain_scr_2 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="medial", **brain_kwargs)
for lb, lb_name, color in zip(lbs, label_names, colors):
    if lb.hemi == "lh":
        brain_scr_3 = brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

brain_scr_3 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="medial", **brain_kwargs)
for lb, lb_name, color in zip(lbs, label_names, colors):
    if lb.hemi == "rh":
        brain_scr_4 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)
        
brain_scr_4 = brain.screenshot()
fig, axes = plt.subplots(2, 2, figsize=(9, 7))
fig.subplots_adjust(hspace=0.1)
for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], [brain_scr_1, brain_scr_2, brain_scr_3, brain_scr_4]):
    nonwhite_pix = (brain != 255).any(-1)
    nonwhite_row = nonwhite_pix.any(1)
    nonwhite_col = nonwhite_pix.any(0)
    cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
    ax.imshow(cropped_screenshot)
    ax.axis("off")
fig.savefig(Path.cwd().parent / "figures" / "brains_aparc.pdf")

###### fig 3_B ######
fname = Path.cwd().parent / "data" / "connections.csv"
df = pd.read_csv(fname)
df_alpha_1 = df[(df["freq_band"]=="alpha") & (df["label_2"]=="superiorfrontal-lh")]
df_alpha_2 = df[(df["freq_band"]=="alpha") & (df["label_2"]=="superiorfrontal-rh")]

df_beta_1 = df[(df["freq_band"]=="beta") & (df["label_2"]=="rostralmiddlefrontal-lh")]
df_beta_2 = df[(df["freq_band"]=="beta") & (df["label_2"]=="lateraloccipital-lh")]

df_gamma_1 = df[(df["freq_band"]=="gamma") & (df["label_2"]=="parsorbitalis-lh")]
df_gamma_2 = df[(df["freq_band"]=="gamma") & (df["label_2"]=="superiorfrontal-lh")]
df_gamma_3 = df[(df["freq_band"]=="gamma") & (df["label_2"]=="superiorfrontal-rh")]

dfs = [df_alpha_1, df_alpha_2, df_beta_1, df_beta_2, df_gamma_1, df_gamma_2, df_gamma_3]
palette_color = ['#1f77b4', '#d62728']
hue_order = ["control", "case"]


for serie, title in zip([range(2), range(2, 4), range(4, 7)], ["alpha", "beta", "gamma"]):
    if title == "gamma":
        fig, axs = plt.subplots(1, 3, figsize=(5, 3))
    else:
        fig, axs = plt.subplots(1, 2, figsize=(5, 3))
    
    for ax_idx, (idx, ax) in enumerate(zip(serie, axs)):
        sns.boxplot(data=dfs[idx], x="group", y="connection_strength", hue="group",
                        hue_order=hue_order, palette=palette_color, fill=False, width=0.6,
                        gap=0, linewidth=1.8, ax=ax)
        sns.stripplot(data=dfs[idx], x="group", y="connection_strength", hue="group", 
                        hue_order=hue_order, palette=palette_color, linewidth=0, size=3.5,
                        edgecolor=None, ax=ax)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_ylabel("")
        ax.set_ylim([None, 0.04])
        if ax_idx in [1, 2]:
            ax.spines['left'].set_visible(False)
            ax.set_yticks([])

    # fig.savefig(Path.cwd().parent / "figures" / f"{title}.pdf")

dfs = [df_alpha_1, df_alpha_2, df_beta_1, df_beta_2, df_gamma_1, df_gamma_2, df_gamma_3]
def plot_glass_brains(dfs, title):
    ###### fig 3_C ######
    labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc', subjects_dir=None, verbose=False)[:-1]
    node_coords = []
    for label in labels:
        if label.hemi == 'lh':
            hemi = 0
        if label.hemi == 'rh':
            hemi = 1
        center_vertex = label.center_of_mass(subject='fsaverage', 
                                            restrict_vertices=False, 
                                            subjects_dir=None)
        mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                                subject='fsaverage', subjects_dir=None)
        node_coords.append(mni_pos)
    node_coords = np.array(node_coords)
    ticks = [lb.name for lb in labels]
    matrix = np.zeros(shape=(len(labels), len(labels)))
    for df in dfs:
        idx1 = ticks.index(df["label_1"].unique()[0])
        idx2 = ticks.index(df["label_2"].unique()[0])
        matrix[idx1][idx2] = 1 

    graph = matrix + matrix.T
    custom_cmap = sns.cubehelix_palette(10, rot=-.45, light=.65, reverse=True, as_cmap=True)
    fig, ax = plt.subplots(1, 1, figsize=(11, 3))
    edge_kwargs = {"lw": 3}
    plot_connectome(adjacency_matrix=graph, node_coords=node_coords, display_mode="lzry", edge_cmap=custom_cmap,
                    node_color='k', node_size=10, axes=ax, colorbar=False,
                    edge_threshold="90%", edge_kwargs=edge_kwargs)
    fig.tight_layout()
    fig.savefig(Path.cwd().parent / "figures" / f"{title}_glass_brain_green.pdf")

for sub_dfs, title in zip([dfs[:2], dfs[2:4], dfs[4:]], ["alpha", "beta", "gamma"]):
    plot_glass_brains(sub_dfs, title)

In [ ]:
###### fig 4_A ###### 

fname = Path.cwd().parent / "data" / "icc_obs_controls.npy"
icc_obs_controls = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "icc_obs_cases.npy"
icc_obs_cases = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "surrogate_unspecific_group_10_random.npy"
icc_surr = np.load(fname, allow_pickle=True)

## compute nodal brain fingerprint per group
n_edges = 68
p_values_control = np.zeros(shape=(n_edges, n_edges))
p_values_case = np.zeros(shape=(n_edges, n_edges))

for i in range(n_edges):
    for j in range(n_edges):
        count_extreme_1 = np.sum(icc_surr[:, i, j] >= icc_obs_controls[i, j])
        p_values_control[i, j] = (count_extreme_1 + 1)  / (len(icc_surr) + 1) ## add +1

        count_extreme_2 = np.sum(icc_surr[:, i, j] >= icc_obs_cases[i, j])
        p_values_case[i, j] = (count_extreme_2 + 1) / (len(icc_surr) + 1) ## add +1

threshold = 0.05
binary_mask_control = (p_values_control < threshold).astype(int) # less and equal
binary_mask_case = (p_values_case < threshold).astype(int) # less and equal

sig_icc_control = icc_obs_controls * binary_mask_control
sig_icc_case = icc_obs_cases * binary_mask_case

df_icc_control = pd.DataFrame(icc_obs_cases).stack().reset_index()
df_icc_control.rename(columns={"level_0" : "lb-1", "level_1": "lb-2", 0: "ICC-val"}, inplace=True)

sns.set_theme(style="whitegrid")
pal = sns.diverging_palette(230, 20, as_cmap=True)
g = sns.relplot(data=df_icc_control, x="lb-1", y="lb-2", hue="ICC-val",
                palette=pal, hue_norm=(0, 1), sizes=(1, 1), legend=False,
                height=5, aspect=1)
g.ax.invert_yaxis()
g.set(xlabel="", ylabel="", xticklabels="", yticklabels="")
g.despine(top=True, right=True, left=True, bottom=True)
g.ax.margins(0.02)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "icc_obs_cases.pdf")

###### fig 4_B ###### 
def plot_4_brains(bl_idxs, nodes_strength):
    brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))
    brain_labels_sub = list(np.array(brain_labels[bl_idxs]))
    nodes_strength_norm = (nodes_strength - nodes_strength.min()) / (nodes_strength.max() - nodes_strength.min())
    brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])

    pal = sns.color_palette("ch:s=.25,rot=-.25", as_cmap=False, n_colors=100).as_hex()
    colors = [pal[int(value * (len(pal) - 1))] for value in nodes_strength_norm]

    brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="lateral", **brain_kwargs)
    for lb, color in zip(brain_labels_sub, colors):
        if lb.hemi == "lh":
            brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

    brain_scr_1 = brain.screenshot()
    brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="lateral", **brain_kwargs)
    for lb, color in zip(brain_labels_sub, colors):
        if lb.hemi == "rh":
            brain_scr_2 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)

    brain_scr_2 = brain.screenshot()
    brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="medial", **brain_kwargs)
    for lb, color in zip(brain_labels_sub, colors):
        if lb.hemi == "lh":
            brain_scr_3 = brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

    brain_scr_3 = brain.screenshot()
    brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="medial", **brain_kwargs)
    for lb, color in zip(brain_labels_sub, colors):
        if lb.hemi == "rh":
            brain_scr_4 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)

    brain_scr_4 = brain.screenshot()
    fig, axes = plt.subplots(2, 2, figsize=(9, 7))
    fig.subplots_adjust(hspace=0.1)
    for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], [brain_scr_1, brain_scr_2, brain_scr_3, brain_scr_4]):
        nonwhite_pix = (brain != 255).any(-1)
        nonwhite_row = nonwhite_pix.any(1)
        nonwhite_col = nonwhite_pix.any(0)
        cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
        ax.imshow(cropped_screenshot)
        ax.axis("off")
    
    return fig, [bl.name for bl in brain_labels_sub]

Clustering

In [ ]:
## function for plotting ellipse
def plot_gaussian_ellipse(mean, cov, ax, n_std=2.0, **kwargs):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    width, height = 2 * n_std * np.sqrt(vals)
    ellip = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    ax.add_patch(ellip)

## loading IM matrixes
fname = Path.cwd().parent / "data" / "identifiability_no_subtract.pkl"
with open(fname, 'rb') as file:
    im_dict = pickle.load(file)

## make the IM matrix symmetric
labels = ["control"] * 23 + ["case"] * 18
im_matrix = im_dict["alpha"]["aparc"][1] # selecting alpha for now
s_col = cosine_similarity(im_matrix.T)
s_row = cosine_similarity(im_matrix)
sim_matrix = 0.5 * (s_col + s_row)

## clustering and plotting dendrogram
Z = linkage(sim_matrix, method='ward')
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
dendrogram(Z, p=30, labels=labels, ax=ax)
ax.set(frame_on=False, yticks=[], title="Hierarchical Clustering Dendrogram")

mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_2d = mds.fit_transform(1 - sim_matrix)

fig, axs = plt.subplots(1, 4, figsize=(14, 4))
criterion = "maxclust"
m_values = range(2, 6)
colors = ["blue"] * 23 + ["red"] * 18
for m, ax in zip(m_values, axs):
    clusters = fcluster(Z, m, criterion=criterion)
    silh_hier_avg = silhouette_score(sim_matrix, clusters, metric='euclidean')
    
    for cluster in np.unique(clusters):
        cluster_points = X_2d[clusters == cluster]
        mean = np.mean(cluster_points, axis=0)
        cov = np.cov(cluster_points.T)
        ax.scatter(X_2d[:, 0], X_2d[:, 1], c=colors, s=15)
        plot_gaussian_ellipse(mean, cov, ax, n_std=2, alpha=0.15, color=np.random.rand(3,))
        ax.spines[["right", "top"]].set_visible(False)
        if m != 2:
            ax.set(yticks=[])
        ax.set_title(f"M = {m} \n avg silhouette score = {round(silh_hier_avg, 4)}", fontsize=9)

blue_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Controls')
red_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Cases')
ax.legend(handles=[blue_patch, red_patch], frameon=False)

In [ ]:
## plotting the same for spectral clustering
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_2d = mds.fit_transform(1 - sim_matrix)

m_values = range(2, 6)
fig, axs = plt.subplots(1, 4, figsize=(14, 4))
for m, ax in zip(m_values, axs):
    spectral = SpectralClustering(n_clusters=m, affinity="precomputed", random_state=42)
    clusters = spectral.fit_predict(sim_matrix)
    silh_spect_avg = silhouette_score(sim_matrix, clusters, metric='euclidean')
    
    for cluster in np.unique(clusters):
        cluster_points = X_2d[clusters == cluster]
        mean = np.mean(cluster_points, axis=0)
        cov = np.cov(cluster_points.T)
        ax.scatter(X_2d[:, 0], X_2d[:, 1], c=colors, s=15)
        plot_gaussian_ellipse(mean, cov, ax, n_std=2, alpha=0.15, color=np.random.rand(3,))
        ax.spines[["right", "top"]].set_visible(False)
        if m != 2:
            ax.set(yticks=[])
        ax.set_title(f"M = {m} \n avg silhouette score = {round(silh_spect_avg, 4)}", fontsize=9)

blue_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Controls')
red_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Cases')
ax.legend(handles=[blue_patch, red_patch], frameon=False)

In [ ]:
## what if I use gaussian mixture in 1d and 2d with 3 components
im_matrix = im_dict[title]["aparc"][1]
diagonal_values = np.diag(im_matrix)
off_diagonal_averages = (np.sum(im_matrix, axis=1) - diagonal_values) / (im_matrix.shape[1] - 1)
im_diff = diagonal_values - off_diagonal_averages

rnd = 42 # 1
gmm = GaussianMixture(n_components=3, random_state=rnd)  
gmm.fit(im_diff.reshape(-1, 1))
print(f"AIC: {round(gmm.aic(im_diff.reshape(-1, 1)), 2)}")
print(f"BIC: {round(gmm.bic(im_diff.reshape(-1, 1)), 2)}")

x = np.linspace(0, 1, 1000)
logprob = gmm.score_samples(x.reshape(-1, 1))
pdf = np.exp(logprob)

kwargs = {"linewidth": 2}
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.kdeplot(im_diff, fill=True, color="black", alpha=0.1, ax=ax, **kwargs)
for mean, cov, weight in zip(gmm.means_, gmm.covariances_, gmm.weights_):
    std = np.sqrt(cov)
    component_pdf = weight * norm.pdf(x, mean, std)
    ax.plot(x, component_pdf.T, linestyle='--', label=f'Component (mean={mean[0]:.2f})', **kwargs)

ax.set(title = "1D distribution of I_diff", xlim=[0, 1])
ax.legend(frameon=False)

gmm = GaussianMixture(n_components=3, random_state=rnd) 
gmm.fit(X_2d)
labels = ["control"] * 23 + ["case"] * 18
means = gmm.means_
covariances = gmm.covariances_

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.scatterplot(x=X_2d[:, 0], y=X_2d[:, 1], hue=labels, palette=["blue", "red"], s=50, edgecolor='k', legend=True, ax=ax)
for mean, cov in zip(means, covariances):
    plot_gaussian_ellipse(mean, cov, ax, n_std=2, alpha=0.15, color=np.random.rand(3,))

ax.legend(frameon=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_title("2D distribution of I_diff")

In [ ]:
dir1 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1'
dir2 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3'

sfreq = 250
end_sample = 5 * 60 * sfreq 
dist_type = "sqeuclidean"
alpha = 1
step = 0.5
max_iter = 10000
rtol = 1e-16
beta = 1
freq_range = 'alpha'

fnames = []
for dir in [dir1, dir2]:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'aparc_mean.npy') and freq_range in f:
            fnames.append(f)

graphs = []
for fname in tqdm(fnames):
    tcs = np.load(file=fname, allow_pickle=True)[:, :end_sample]
    graph = learn_graph.log_degree_barrier(X=tcs, dist_type=dist_type,
                                                alpha=alpha, beta=beta, step=step,
                                                w0=None, maxit=max_iter, rtol=rtol,
                                                retall=False)
    graph = graph / np.linalg.norm(graph, 'fro')
    graph[graph < 0.01 * np.max(graph)] = 0
    graphs.append(graph)

graphs = np.array(graphs)

labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc",
                                    subjects_dir=None, verbose=False)[:-1]
node_coords = []
for label in labels:
    if label.hemi == 'lh': hemi = 0
    if label.hemi == 'rh': hemi = 1
    center_vertex = label.center_of_mass(subject="fsaverage", 
                                        restrict_vertices=False, 
                                        subjects_dir=None)
    mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                                subject="fsaverage", subjects_dir=None)
    node_coords.append(mni_pos)

controls_array = np.array([i for i in range(23) if i not in {9, 19, 1, 18}])
cases_array = np.array([i for i in range(23, 41) if i not in {36, 24, 31}])

node_coords = np.array(node_coords)
ticks = [lb.name for lb in labels]
fig, axs = plt.subplots(2, 1, figsize=(11, 5))
for ids, ax in zip([controls_array, cases_array], axs):
    avg_graph = graphs[ids].mean(axis=0)
    plot_connectome(adjacency_matrix=avg_graph, node_coords=node_coords,
                    node_color='k', node_size=10, axes=ax,
                    edge_threshold='98%')
fig.tight_layout()

In [ ]:
## is MSD representation as a sort of ground truth? the answer is yes.
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_2d = mds.fit_transform(1 - sim_matrix)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
spectral = SpectralClustering(n_clusters=3, affinity="precomputed", random_state=42)
clusters = spectral.fit_predict(sim_matrix)

# spectral = KMeans(n_clusters=3, random_state=5)
# clusters = spectral.fit_predict(X_2d)

color_mapping = {0: 'olive', 1: 'darkviolet', 2: 'coral'}
point_idx = 31

for cluster in np.unique(clusters):
    cluster_points = X_2d[clusters == cluster]
    mean = np.mean(cluster_points, axis=0)
    cov = np.cov(cluster_points.T)
    # colors = np.array([color_mapping[value] for value in clusters])
    colors = ["cyan"] * 23 + ["red"] * 18
    ax.scatter(X_2d[:, 0], X_2d[:, 1], c=colors, s=30)
    ax.scatter(X_2d[point_idx, 0], X_2d[point_idx, 1], c="k", s=30)
    plot_gaussian_ellipse(mean, cov, ax, n_std=2, alpha=0.08, color=np.random.rand(3,))
    ax.spines[["right", "top"]].set_visible(False)
    if m != 2:
        ax.set(yticks=[])
    ax.set_title(f"M = 3", fontsize=9)

cl1_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='olive', markersize=10, label='cluster_1')
cl2_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='darkviolet', markersize=10, label='cluster_2')
cl3_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='coral', markersize=10, label='cluster_3')

ax.legend(handles=[cl1_patch, cl2_patch, cl3_patch], frameon=False)

straightforward ICC approach, coupled with a nice bootstrapping step

In [ ]:
## create test_retest FCs per subject
dir1 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1'
dir2 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3'
freq_range = 'gamma'
fnames = []
for dir in [dir1, dir2]:
    for filename in sorted(os.listdir(dir)): 
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f.endswith(f'aparc_mean.npy') and freq_range in f:
            fnames.append(f)

n_controls = 23
sfreq = 250
end_sample = 5 * 60 * sfreq 
kwargs = {
        "dist_type": "sqeuclidean",
        "alpha": 1,
        "step": 0.5,
        "maxit": 10000,
        "rtol": 1e-16,
        "beta": 1,
        "w0": None,
        "retall": False
        }

graphs = []
for fname in tqdm(fnames):
    tcs_1 = np.load(file=fname, allow_pickle=True)[:, :int(end_sample/2)]
    tcs_2 = np.load(file=fname, allow_pickle=True)[:, int(end_sample/2):end_sample]
    graph_1 = learn_graph.log_degree_barrier(X=tcs_1, **kwargs)
    graph_2 = learn_graph.log_degree_barrier(X=tcs_2, **kwargs)
    graph_1 = graph_1 / np.linalg.norm(graph_1, 'fro')
    graph_2 = graph_2 / np.linalg.norm(graph_2, 'fro')
    graphs.append([graph_1, graph_2])

graphs_control = np.array(graphs[:n_controls]) # the shape should be (23, 2, 68, 68)
graphs_cases = np.array(graphs[n_controls:])
n_edges = graphs_control.shape[-1]

In [ ]:
## computing observed ICC for controls
mask = np.triu(np.ones((n_edges, n_edges), dtype=bool), k=1)
df_matrix = graphs_control[:, :, mask]
icc_vals = []

for edge_idx in range(df_matrix.shape[-1]):
    df = pd.DataFrame(df_matrix[:,:,edge_idx])
    df_long = df.stack().reset_index()
    df_long.columns = ['measurements', 'judges', 'ratings']
    df_long['measurements'] += 1  
    df_long['judges'] += 1
    icc_df = intraclass_corr(data=df_long, targets='measurements', raters='judges', ratings='ratings')
    icc_val = icc_df.loc[icc_df['Type'] == 'ICC1', 'ICC'].values[0]
    icc_vals.append(icc_val)

icc_matrix = np.zeros(shape=(n_edges, n_edges))
icc_matrix[np.triu_indices(n_edges, k=1)] = np.array([icc_vals])
icc_obs_controls = icc_matrix + icc_matrix.T
np.save("/Users/payamsadeghishabestari/codes/graph-learning/data/icc_obs_controls_gamma.npy", np.array(icc_obs_controls))

## computing observed ICC for cases
mask = np.triu(np.ones((n_edges, n_edges), dtype=bool), k=1)
df_matrix = graphs_cases[:, :, mask]
icc_vals = []

for edge_idx in range(df_matrix.shape[-1]):
    df = pd.DataFrame(df_matrix[:,:,edge_idx])
    df_long = df.stack().reset_index()
    df_long.columns = ['measurements', 'judges', 'ratings']
    df_long['measurements'] += 1  
    df_long['judges'] += 1
    icc_df = intraclass_corr(data=df_long, targets='measurements', raters='judges', ratings='ratings')
    icc_val = icc_df.loc[icc_df['Type'] == 'ICC1', 'ICC'].values[0]
    icc_vals.append(icc_val)

icc_matrix = np.zeros(shape=(n_edges, n_edges))
icc_matrix[np.triu_indices(n_edges, k=1)] = np.array([icc_vals])
icc_obs_cases = icc_matrix + icc_matrix.T
np.save("/Users/payamsadeghishabestari/codes/graph-learning/data/icc_obs_cases_gamma.npy", np.array(icc_obs_cases))


In [ ]:
## create surrogate unspecific group
n_bootstrap = 500
n_selected = 10
mask = np.triu(np.ones((n_edges, n_edges), dtype=bool), k=1)
icc_matrixes = []

for bootstrap in tqdm(range(n_bootstrap)):
    ## make a correct form of matrix for dataframe
    icc_vals = []
    randam_subjects_1 = np.random.choice(range(len(graphs_control)), n_selected, replace=True)
    randam_subjects_2 = np.random.choice(range(len(graphs_cases)), n_selected, replace=True)
    df_matrix_1 = graphs_control[randam_subjects_1][:, :, mask] # shape (n_measurements, n_judges, n_edges)
    df_matrix_2 = graphs_cases[randam_subjects_2][:, :, mask] # shape (n_measurements, n_judges, n_edges)
    df_matrix = np.concatenate([df_matrix_1, df_matrix_2], axis=0)

    ## make a dataframe for ICC and compute it
    for edge_idx in range(df_matrix.shape[-1]):
        df = pd.DataFrame(df_matrix[:,:,edge_idx])
        df_long = df.stack().reset_index()
        df_long.columns = ['measurements', 'judges', 'ratings']
        df_long['measurements'] += 1  
        df_long['judges'] += 1
        icc_df = intraclass_corr(data=df_long, targets='measurements', raters='judges', ratings='ratings')
        icc_val = icc_df.loc[icc_df['Type'] == 'ICC1', 'ICC'].values[0]
        icc_vals.append(icc_val)

    icc_matrix = np.zeros(shape=(n_edges, n_edges))
    icc_matrix[np.triu_indices(n_edges, k=1)] = np.array([icc_vals])
    icc_matrix = icc_matrix + icc_matrix.T
    icc_matrixes.append(icc_matrix)

np.save("/Users/payamsadeghishabestari/codes/graph-learning/data/surrogate_unspecific_group_10_random.npy", np.array(icc_matrixes))

if you are interested in significant difference between controls and cases

In [ ]:
fname = Path.cwd().parent / "data" / "icc_obs_controls.npy"
icc_obs_controls = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "icc_obs_cases.npy"
icc_obs_cases = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "surrogate_unspecific_group_10_random.npy"
icc_surr = np.load(fname, allow_pickle=True)

## compute nodal brain fingerprint per group
p_values_control = np.zeros(shape=(n_edges, n_edges))
p_values_case = np.zeros(shape=(n_edges, n_edges))

for i in range(n_edges):
    for j in range(n_edges):
        count_extreme_1 = np.sum(icc_surr[:, i, j] >= icc_obs_controls[i, j])
        p_values_control[i, j] = (count_extreme_1 + 1)  / (len(icc_surr) + 1) ## add +1

        count_extreme_2 = np.sum(icc_surr[:, i, j] >= icc_obs_cases[i, j])
        p_values_case[i, j] = (count_extreme_2 + 1) / (len(icc_surr) + 1) ## add +1

threshold = 0.05
binary_mask_control = (p_values_control < threshold).astype(int) # less and equal
binary_mask_case = (p_values_case < threshold).astype(int) # less and equal

sig_icc_control = icc_obs_controls * binary_mask_control
sig_icc_case = icc_obs_cases * binary_mask_case

df_icc_control = pd.DataFrame(sig_icc_case).stack().reset_index()
df_icc_control.rename(columns={"level_0" : "lb-1", "level_1": "lb-2", 0: "ICC-val"}, inplace=True)

sns.set_theme(style="whitegrid")
pal = sns.diverging_palette(230, 20, as_cmap=True)
g = sns.relplot(data=df_icc_control, x="lb-1", y="lb-2", hue="ICC-val",
                palette=pal, hue_norm=(0, 1), sizes=(1, 1), legend=False,
                height=5, aspect=1)
g.ax.invert_yaxis()
g.set(xlabel="", ylabel="", xticklabels="", yticklabels="")
g.despine(top=True, right=True, left=True, bottom=True)
g.ax.margins(0.02)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "icc_obs_controls.pdf")

nodes_strength = sig_icc_case.sum(axis=0)
threshold = np.percentile(nodes_strength, 75)
len_nonzero_nodes = len(nodes_strength[nodes_strength > 0])
nodes_strength_sorted = np.argsort(nodes_strength)[::-1][:len_nonzero_nodes]
nodes_strength_sub = nodes_strength[nodes_strength_sorted]
fig, bl_names = plot_4_brains(nodes_strength_sorted, nodes_strength_sub)
fig.savefig(Path.cwd().parent / "figures" / "icc_brain_cases_gamma.pdf")


Picture 5

In [ ]:
fname = Path.cwd().parent / "data" / "icc_obs_controls.npy"
icc_obs_controls_alpha = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "icc_obs_cases.npy"
icc_obs_cases_alpha = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "icc_obs_controls_gamma.npy"
icc_obs_controls_gamma = np.load(fname, allow_pickle=True)
fname = Path.cwd().parent / "data" / "icc_obs_cases_gamma.npy"
icc_obs_cases_gamma = np.load(fname, allow_pickle=True)

items = [icc_obs_controls_alpha, icc_obs_cases_alpha, icc_obs_controls_gamma, icc_obs_cases_gamma]
titles = ["control_alpha", "case_alpha", "control_gamma", "case_gamma"]

## plotting the ICC matrixes
sns.set_theme(style="whitegrid")
for item, title in zip(items, titles): 
    df_icc = pd.DataFrame(item).stack().reset_index()
    df_icc.rename(columns={"level_0" : "lb-1", "level_1": "lb-2", 0: "ICC-val"}, inplace=True)

    pal = sns.diverging_palette(230, 20, as_cmap=True)
    g = sns.relplot(data=df_icc, x="lb-1", y="lb-2", hue="ICC-val",
                    palette=pal, hue_norm=(0, 1), sizes=(1, 1), legend=False,
                    height=5, aspect=1)
    g.ax.invert_yaxis()
    g.set(xlabel="", ylabel="", xticklabels="", yticklabels="")
    g.despine(top=True, right=True, left=True, bottom=True)
    g.ax.margins(0.02)
    g.tight_layout()
    g.savefig(Path.cwd().parent / "figures" / f"icc_{title}.pdf")

## plot the nodal strength
for item, title in zip(items[3:4], titles[3:4]):
    nodes_strength = item.sum(axis=0)
    threshold = np.percentile(nodes_strength, 75)
    len_nonzero_nodes = len(nodes_strength[nodes_strength > threshold])
    nodes_strength_sorted = np.argsort(nodes_strength)[::-1][:len_nonzero_nodes]
    nodes_strength_sub = nodes_strength[nodes_strength_sorted]
    fig, bl_names = plot_4_brains(nodes_strength_sorted, nodes_strength_sub)
    # fig.savefig(Path.cwd().parent / "figures" / f"icc_brain_{title}.pdf")

## plot the distributions
my_dict = {}
n_edges = 68
mask = np.triu(np.ones((n_edges, n_edges), dtype=bool), k=1)
my_dict["icc_values"] = np.concatenate([items[0][mask], items[1][mask], items[2][mask], items[3][mask]], axis=0)
my_dict["group"] = mask.sum() * ["controls"] + mask.sum() * ["cases"] + mask.sum() * ["controls"] + mask.sum() * ["cases"]
my_dict["frequency"] = mask.sum() * ["alpha"] * 2 + mask.sum() * ["gamma"] * 2
df = pd.DataFrame(my_dict)

g = sns.FacetGrid(data=df, col="frequency", height=3, aspect=1.5, xlim=[0, 1], legend_out=False)
pal = ['#1f77b4', '#d62728']
kwargs = {"x": "icc_values", "hue": "group", "stat": "percent", "linewidth": 0.2,
            "palette": pal, "fill": True, "edgecolor": "k"}
g.map_dataframe(sns.histplot, line_kws={"lw": 6}, **kwargs)
legend_data = {"control": mpatches.Patch(color=pal[0]), "case": mpatches.Patch(color=pal[1])} 
g.add_legend(legend_data=legend_data, frameon=False)
g.tight_layout()
g.savefig(Path.cwd().parent / "figures" / "icc_dist.pdf")

Normality check fixed

In [ ]:
my_dict = {"freq_band": [], "label_1": [], "label_2": [], "group": [], "connection_strength": []}

brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False)[:-1]

for freq_range in ["theta", "alpha", "gamma"]:
    ## check normality
    dir1 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg1'
    dir2 = '/Users/payamsadeghishabestari/meg_gsp/stc_labels/tinmeg3'
    fnames = []
    for dir in [dir1, dir2]:
        for filename in sorted(os.listdir(dir)): 
            f = os.path.join(dir, filename)
            if os.path.isfile(f) and f.endswith(f'aparc_mean.npy') and freq_range in f:
                fnames.append(f)

    n_controls = 23
    sfreq = 250
    end_sample = 5 * 60 * sfreq 
    kwargs = {
            "dist_type": "sqeuclidean",
            "alpha": 1,
            "step": 0.5,
            "maxit": 10000,
            "rtol": 1e-16,
            "beta": 1,
            "w0": None,
            "retall": False
            }

    graphs = []
    for fname in tqdm(fnames):
        tcs = np.load(file=fname, allow_pickle=True)[:, :end_sample]
        graph = learn_graph.log_degree_barrier(X=tcs, **kwargs)
        graph = graph / np.linalg.norm(graph, 'fro')
        graphs.append(graph)

    graphs_control = np.array(graphs[:n_controls]) # the shape should be (23, 2, 68, 68)
    graphs_cases = np.array(graphs[n_controls:])
    n_edges = graphs_control.shape[-1]

    mask = np.triu(np.ones((n_edges, n_edges), dtype=bool), k=1)
    up_tri_idxs = np.triu_indices(n_edges, k=1)
    controls_upper = graphs_control[:, mask]
    cases_upper = graphs_cases[:, mask]

    pvals = []
    for i in range(controls_upper.shape[1]):
        _, p_val_control = shapiro(controls_upper[:, i])
        _, p_val_case = shapiro(cases_upper[:, i])

        if p_val_control < 0.05 or p_val_case < 0.05:
            stat, p_val = mannwhitneyu(controls_upper[:, i], cases_upper[:, i])
        
        else:
            stat, p_val = ttest_ind(controls_upper[:, i], cases_upper[:, i])

        pvals.append(p_val)

    reject_null, p_corrected, _, _ = sm.stats.multipletests(pvals=pvals, alpha=0.05,
                                                                method="fdr_bh")

    label_idxs = []
    if len(np.where(p_corrected < 0.05)[0]) == 0:
        print(f"No statistical difference between brain labels for method")
    else:
        for idx in np.where(p_corrected < 0.05)[0]:
            label_idxs.append((up_tri_idxs[0][idx], up_tri_idxs[1][idx]))
            idx_1 = up_tri_idxs[0][idx]
            idx_2 = up_tri_idxs[1][idx]

            for subject in graphs_control:
                conn = subject[idx_1][idx_2]

                my_dict["freq_band"].append(freq_range)
                my_dict["label_1"].append(brain_labels[idx_1].name)
                my_dict["label_2"].append(brain_labels[idx_2].name)
                my_dict["group"].append("control")
                my_dict["connection_strength"].append(conn)

            for subject in graphs_cases:
                conn = subject[idx_1][idx_2]

                my_dict["freq_band"].append(freq_range)
                my_dict["label_1"].append(brain_labels[idx_1].name)
                my_dict["label_2"].append(brain_labels[idx_2].name)
                my_dict["group"].append("case")
                my_dict["connection_strength"].append(conn)
                
df_1 = pd.DataFrame(my_dict)

In [ ]:
## plotting the boxplots
fname = Path.cwd().parent / "data" / "stat_comp.csv"
df = pd.read_csv(fname)
df["labels_together"] = df["label_1"] + " & " + df["label_2"]
df_theta = df.query('freq_band == "theta"')
df_alpha = df.query('freq_band == "alpha"')
df_gamma = df.query('freq_band == "gamma"')
palette_color = ['#1f77b4', '#d62728'] 
hue_order = ["control", "case"]

fig, axs = plt.subplots(3, 1, figsize=(5.5, 8))

box_kwargs = {"width": .8, "gap": 0.3, "linewidth": 1.5}
size = 4
sns.boxplot(data=df_theta, x="labels_together", y="connection_strength", hue="group", 
            hue_order=hue_order, palette=palette_color, fill=False, ax=axs[0], legend=False, **box_kwargs)
sns.stripplot(data=df_theta, x="labels_together", y="connection_strength", hue="group", dodge=True,
                hue_order=hue_order, palette=palette_color, size=size,
                edgecolor=None, ax=axs[0], legend=False)


box_kwargs = {"width": 0.8, "gap": 0.3, "linewidth": 1.4}
size = 2.8
sns.boxplot(data=df_alpha, x="labels_together", y="connection_strength", hue="group", 
            hue_order=hue_order, palette=palette_color, fill=False, ax=axs[1], legend=False, **box_kwargs)
sns.stripplot(data=df_alpha, x="labels_together", y="connection_strength", hue="group", dodge=True,
                hue_order=hue_order, palette=palette_color, size=size,
                edgecolor=None, ax=axs[1], legend=False)

box_kwargs = {"width": 0.8, "gap": 0.3, "linewidth": 1.3}
size = 2.4

sns.boxplot(data=df_gamma, x="labels_together", y="connection_strength", hue="group", 
            hue_order=hue_order, palette=palette_color, fill=False, ax=axs[2], legend=False, **box_kwargs)
sns.stripplot(data=df_gamma, x="labels_together", y="connection_strength", hue="group", dodge=True,
                hue_order=hue_order, palette=palette_color, size=size,
                edgecolor=None, ax=axs[2], legend=False)

for ax in axs:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel("")
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=4)
    ax.set_ylim([-0.005, 0.04])
    ax.set_xlabel("")

fig.tight_layout()
fig.savefig(Path.cwd().parent / "figures" / f"new_boxplots.pdf")

## plotting the glass brains
def plot_glass_brains(df, title):
    ###### fig 3_C ######
    labels = mne.read_labels_from_annot(subject='fsaverage', parc='aparc', subjects_dir=None, verbose=False)[:-1]
    node_coords = []
    for label in labels:
        if label.hemi == 'lh':
            hemi = 0
        if label.hemi == 'rh':
            hemi = 1
        center_vertex = label.center_of_mass(subject='fsaverage', 
                                            restrict_vertices=False, 
                                            subjects_dir=None)
        mni_pos = mne.vertex_to_mni(center_vertex, hemis=hemi,
                                subject='fsaverage', subjects_dir=None)
        node_coords.append(mni_pos)
    node_coords = np.array(node_coords)
    ticks = [lb.name for lb in labels]
    matrix = np.zeros(shape=(len(labels), len(labels)))
    
    for lb in df["labels_together"].unique():
        print(lb)
        df_1 = df.query(f'labels_together == "{lb}"')
        idx1 = ticks.index(df_1["label_1"].unique()[0])
        idx2 = ticks.index(df_1["label_2"].unique()[0])
            
            
        matrix[idx1][idx2] = 1 

    graph = matrix + matrix.T
    custom_cmap = sns.cubehelix_palette(10, rot=-.45, light=.65, reverse=True, as_cmap=True)
    fig, ax = plt.subplots(1, 1, figsize=(11, 3))
    edge_kwargs = {"lw": 3}
    plot_connectome(adjacency_matrix=graph, node_coords=node_coords, display_mode="lzry", edge_cmap=custom_cmap,
                    node_color='k', node_size=10, axes=ax, colorbar=False,
                    edge_threshold="90%", edge_kwargs=edge_kwargs)
    fig.tight_layout()
    fig.savefig(Path.cwd().parent / "figures" / f"new_{title}_glass_brain_green.pdf")

for sub_dfs, title in zip([df_theta, df_alpha, df_gamma][2:3], ["theta", "alpha", "gamma"][2:3]):
    plot_glass_brains(sub_dfs, title)

In [5]:
label_names = ["fusiform-rh", "bankssts-rh", "cuneus-lh", "lateraloccipital-lh",
                "paracentral-rh", "precentral-lh", "inferiorparietal-lh",
                "precuneus-lh", "insula-lh", "parahippocampal-lh",
                "lateralorbitofrontal-lh", "transversetemporal-lh",
                "superiorfrontal-lh", "superiorfrontal-rh",
                "bankssts-lh", "parsorbitalis-lh", "frontalpole-lh",
                "frontalpole-rh", "paracentral-lh",
                "superiorparietal-lh"][4:5]

brain_labels = np.array(mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=False))
lb_names = [lb.name for lb in brain_labels]
idxs = np.array([lb_names.index(lb) for lb in label_names])
lbs = list(brain_labels[idxs])
cl1 = sns.cubehelix_palette(10, rot=2.5, light=.7, reverse=True)[9]
cl2 = sns.cubehelix_palette(10, rot=-2*np.pi/10, light=.7, reverse=True)[1]
cl3 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[3]
cl4 = list(sns.color_palette("Reds")[-1])
cl5 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[2]
cl6 = sns.cubehelix_palette(10, rot=8.5, light=.7, reverse=True)[8]
cl7 = sns.cubehelix_palette(10, rot=8.5, light=.7, reverse=True)[0]
cl8 = sns.cubehelix_palette(10, rot=1.5, light=.7, reverse=True)[2]
cl9 = sns.cubehelix_palette(10, rot=4.5, light=.7, reverse=True)[5]
cl10 = sns.diverging_palette(100, 20, as_cmap=False, n=10)[1]
cl11 = sns.diverging_palette(100, 20, as_cmap=False, n=10)[5]
cl12 = sns.diverging_palette(100, 20, as_cmap=False, n=10)[9]
cl13 = sns.diverging_palette(180, 60, as_cmap=False, n=10)[0]
cl14 = sns.diverging_palette(180, 60, as_cmap=False, n=10)[4]
cl15 = sns.diverging_palette(180, 60, as_cmap=False, n=10)[7]
cl16 = sns.diverging_palette(290, 10, as_cmap=False, n=10)[2]
cl17 = sns.diverging_palette(290, 10, as_cmap=False, n=10)[5]
cl18 = sns.diverging_palette(290, 10, as_cmap=False, n=10)[8]
cl19 = sns.color_palette("icefire", as_cmap=False, n_colors=10)[1]
cl20 = sns.color_palette("icefire", as_cmap=False, n_colors=10)[3]
cl21 = sns.color_palette("icefire", as_cmap=False, n_colors=10)[8]

colors = [cl1, cl2, cl3, cl4, cl5, cl6, cl7, cl8, cl9, cl10, cl11, cl12, cl13, cl14, cl15, cl16, cl17, cl18, cl19, cl20, cl21]

brain_kwargs = dict(background="white", surf="pial_semi_inflated", cortex=["#b8b4ac", "#b8b4ac"])

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="lateral", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "lh":
        brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

brain_scr_1 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="lateral", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "rh":
        brain_scr_2 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)

brain_scr_2 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="lh", views="medial", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "lh":
        brain_scr_3 = brain.add_label(lb, hemi="lh", color=color, borders=False, alpha=0.7)

brain_scr_3 = brain.screenshot()
brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="rh", views="medial", **brain_kwargs)
for lb, color in zip(lbs, colors):
    if lb.hemi == "rh":
        brain_scr_4 = brain.add_label(lb, hemi="rh", color=color, borders=False, alpha=0.7)
        
brain_scr_4 = brain.screenshot()

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="both", views="rostral", **brain_kwargs)
for lb, color in zip(lbs, colors):
        brain_scr_5 = brain.add_label(lb, hemi="both", color=color, borders=False, alpha=0.7)
        
brain_scr_5 = brain.screenshot()

brain = mne.viz.Brain("fsaverage", subjects_dir=None, hemi="both", views="caudal", **brain_kwargs)
for lb, color in zip(lbs, colors):
        brain_scr_6 = brain.add_label(lb, hemi="both", color=color, borders=False, alpha=0.7)
        
brain_scr_6 = brain.screenshot()


fig, axes = plt.subplots(2, 2, figsize=(9, 7))
fig.subplots_adjust(hspace=0.1)
for ax, brain in zip([axes[0][0], axes[0][1], axes[1][0], axes[1][1]], [brain_scr_1, brain_scr_2, brain_scr_3, brain_scr_4]):
    nonwhite_pix = (brain != 255).any(-1)
    nonwhite_row = nonwhite_pix.any(1)
    nonwhite_col = nonwhite_pix.any(0)
    cropped_screenshot = brain[nonwhite_row][:, nonwhite_col]
    ax.imshow(cropped_screenshot)
    ax.axis("off")
# fig.savefig(Path.cwd().parent / "figures" / "brains_aparc_new.pdf")